In [1]:
import os
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt


# Step 1: Load the pre-trained ResNet50 model
model = models.resnet50(pretrained=True)
model.eval()

# Step 2: Load the images
input_directory ="C:/Users/DandaC8719/Documents/capstone_overall_code/output_images_folder(DenseNet)_30"
output_original_directory = "C:/Users/DandaC8719/Documents/capstone_overall_code/original_incorrect_predictions_folder(FGSM)_ResNet_DenseNet"
output_adversarial_directory = "C:/Users/DandaC8719/Documents/capstone_overall_code/adversarial_images_folder(FGSM)_ResNet_DenseNet"

input_images = [os.path.join(input_directory, filename) for filename in os.listdir(input_directory) if filename.endswith(".JPEG")]

# Step 3: Define the FGSM attack function
def fgsm_attack(image, epsilon, data_grad):
    sign_data_grad = data_grad.sign()
    perturbed_image = image + epsilon * sign_data_grad
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    return perturbed_image

# Step 4: Generate adversarial examples
epsilon = 0.01  # Adjust this value as needed
successful_adversarial_examples = []
corres_org_img=[]
count=0
for input_image in input_images[:8000]:
    # Load the original image
    original_image = Image.open(input_image).convert("RGB")
    
    # Convert the PIL image to a PyTorch tensor and apply the necessary preprocessing
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ])
    image_tensor = preprocess(original_image).unsqueeze(0)
    image_tensor.requires_grad = True

    # Forward pass to get the predictions
    output = model(image_tensor)
    original_label = torch.argmax(output, 1).item()

    # Calculate the loss
    loss = nn.CrossEntropyLoss()(output, torch.tensor([original_label]))

    # Zero all existing gradients
    model.zero_grad()

    # Backpropagate the loss to compute gradients
    loss.backward()

    # Collect the gradients of the input image
    data_grad = image_tensor.grad.data

    # Generate the perturbed image using the FGSM attack
    perturbed_image = fgsm_attack(image_tensor, epsilon, data_grad)

    # Forward pass with the perturbed image
    perturbed_output = model(perturbed_image)

    # Check if the perturbed image's predicted label differs from the original label
    perturbed_label = torch.argmax(perturbed_output, 1).item()
    if perturbed_label != original_label:
        # Save the perturbed image as a successful adversarial example
        successful_adversarial_examples.append(perturbed_image)
        corres_org_img.append(input_image)
        # Save the original image to output_original_directory
        os.makedirs(output_original_directory, exist_ok=True)
        original_image_name = os.path.basename(input_image)
        output_original_image_path = os.path.join(output_original_directory, original_image_name)
        original_image.save(output_original_image_path)
        
        # Save the perturbed image to output_adversarial_directory
        os.makedirs(output_adversarial_directory, exist_ok=True)
        perturbed_image_name = f"perturbed_{original_image_name}"
        output_perturbed_image_path = os.path.join(output_adversarial_directory, perturbed_image_name)
        perturbed_image_pil = transforms.ToPILImage()(perturbed_image.squeeze(0))
        perturbed_image_pil.save(output_perturbed_image_path)
        print(f"image count is: {count}")
        count +=1

# Print the length of successful adversarial examples
print("Number of successful adversarial examples:", len(successful_adversarial_examples))
print(len(corres_org_img))
print(count)
# # Show the original and adversarial images for all successful examples
# for idx, perturbed_image in enumerate(successful_adversarial_examples):
#     # Load the original image
#     original_image_path = corres_org_img[idx]
#     try:
#         original_image = Image.open(original_image_path).convert("RGB")
#     except Exception as e:
#         print(f"Error loading original image {original_image_path}: {e}")
#         continue

#     # Convert the perturbed image to a numpy array
#     perturbed_image_np = perturbed_image.squeeze().detach().cpu().numpy().transpose(1, 2, 0)

#     # Get the original label
#     with torch.no_grad():
#         original_image_tensor = preprocess(original_image).unsqueeze(0)
#         original_output = model(original_image_tensor)
#         original_label_idx = torch.argmax(original_output, 1).item()

#     # Get the adversarial label
#     with torch.no_grad():
#         perturbed_output = model(perturbed_image)
#         perturbed_label_idx = torch.argmax(perturbed_output, 1).item()

#     fig, axes = plt.subplots(1, 2, figsize=(10, 5))

#     # Plot the original image
#     axes[0].imshow(original_image)
#     axes[0].set_title(f"Original Image {idx+1}, Label: {original_label_idx}")
#     axes[0].axis('off')

#     # Plot the adversarial image
#     axes[1].imshow(perturbed_image_np)
#     axes[1].set_title(f"Adversarial Image {idx+1}, Label: {perturbed_label_idx}")
#     axes[1].axis('off')

#     plt.show()

C:\Users\DandaC8719\AppData\Local\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\DandaC8719\AppData\Local\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


image count is: 0
image count is: 1
image count is: 2
image count is: 3
image count is: 4
image count is: 5
image count is: 6
image count is: 7
image count is: 8
image count is: 9
image count is: 10
image count is: 11
image count is: 12
image count is: 13
image count is: 14
image count is: 15
image count is: 16
image count is: 17
image count is: 18
image count is: 19
image count is: 20
image count is: 21
image count is: 22
image count is: 23
image count is: 24
image count is: 25
image count is: 26
image count is: 27
image count is: 28
image count is: 29
image count is: 30
image count is: 31
image count is: 32
image count is: 33
image count is: 34
image count is: 35
image count is: 36
image count is: 37
image count is: 38
image count is: 39
image count is: 40
image count is: 41
image count is: 42
image count is: 43
image count is: 44
image count is: 45
image count is: 46
image count is: 47
image count is: 48
image count is: 49
image count is: 50
image count is: 51
image count is: 52
ima

In [2]:
import os
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt


# Step 1: Load the pre-trained ResNet50 model
model = models.resnet50(pretrained=True)
model.eval()

# Step 2: Load the images
input_directory ="C:/Users/DandaC8719/Documents/capstone_overall_code/output_images_folder(DenseNet)_30"
output_original_directory = "C:/Users/DandaC8719/Documents/capstone_overall_code/original_incorrect_predictions_folder(FGSM)_ResNet_DenseNet"
output_adversarial_directory = "C:/Users/DandaC8719/Documents/capstone_overall_code/adversarial_images_folder(FGSM)_ResNet_DenseNet"

input_images = [os.path.join(input_directory, filename) for filename in os.listdir(input_directory) if filename.endswith(".JPEG")]

# Step 3: Define the FGSM attack function
def fgsm_attack(image, epsilon, data_grad):
    sign_data_grad = data_grad.sign()
    perturbed_image = image + epsilon * sign_data_grad
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    return perturbed_image

# Step 4: Generate adversarial examples
epsilon = 0.01  # Adjust this value as needed
successful_adversarial_examples = []
corres_org_img=[]
count=0
for input_image in input_images[8000:16000]:
    # Load the original image
    original_image = Image.open(input_image).convert("RGB")
    
    # Convert the PIL image to a PyTorch tensor and apply the necessary preprocessing
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ])
    image_tensor = preprocess(original_image).unsqueeze(0)
    image_tensor.requires_grad = True

    # Forward pass to get the predictions
    output = model(image_tensor)
    original_label = torch.argmax(output, 1).item()

    # Calculate the loss
    loss = nn.CrossEntropyLoss()(output, torch.tensor([original_label]))

    # Zero all existing gradients
    model.zero_grad()

    # Backpropagate the loss to compute gradients
    loss.backward()

    # Collect the gradients of the input image
    data_grad = image_tensor.grad.data

    # Generate the perturbed image using the FGSM attack
    perturbed_image = fgsm_attack(image_tensor, epsilon, data_grad)

    # Forward pass with the perturbed image
    perturbed_output = model(perturbed_image)

    # Check if the perturbed image's predicted label differs from the original label
    perturbed_label = torch.argmax(perturbed_output, 1).item()
    if perturbed_label != original_label:
        # Save the perturbed image as a successful adversarial example
        successful_adversarial_examples.append(perturbed_image)
        corres_org_img.append(input_image)
        # Save the original image to output_original_directory
        os.makedirs(output_original_directory, exist_ok=True)
        original_image_name = os.path.basename(input_image)
        output_original_image_path = os.path.join(output_original_directory, original_image_name)
        original_image.save(output_original_image_path)
        
        # Save the perturbed image to output_adversarial_directory
        os.makedirs(output_adversarial_directory, exist_ok=True)
        perturbed_image_name = f"perturbed_{original_image_name}"
        output_perturbed_image_path = os.path.join(output_adversarial_directory, perturbed_image_name)
        perturbed_image_pil = transforms.ToPILImage()(perturbed_image.squeeze(0))
        perturbed_image_pil.save(output_perturbed_image_path)
        print(f"image count is: {count}")
        count +=1

# Print the length of successful adversarial examples
print("Number of successful adversarial examples:", len(successful_adversarial_examples))
print(len(corres_org_img))
print(count)
# # Show the original and adversarial images for all successful examples
# for idx, perturbed_image in enumerate(successful_adversarial_examples):
#     # Load the original image
#     original_image_path = corres_org_img[idx]
#     try:
#         original_image = Image.open(original_image_path).convert("RGB")
#     except Exception as e:
#         print(f"Error loading original image {original_image_path}: {e}")
#         continue

#     # Convert the perturbed image to a numpy array
#     perturbed_image_np = perturbed_image.squeeze().detach().cpu().numpy().transpose(1, 2, 0)

#     # Get the original label
#     with torch.no_grad():
#         original_image_tensor = preprocess(original_image).unsqueeze(0)
#         original_output = model(original_image_tensor)
#         original_label_idx = torch.argmax(original_output, 1).item()

#     # Get the adversarial label
#     with torch.no_grad():
#         perturbed_output = model(perturbed_image)
#         perturbed_label_idx = torch.argmax(perturbed_output, 1).item()

#     fig, axes = plt.subplots(1, 2, figsize=(10, 5))

#     # Plot the original image
#     axes[0].imshow(original_image)
#     axes[0].set_title(f"Original Image {idx+1}, Label: {original_label_idx}")
#     axes[0].axis('off')

#     # Plot the adversarial image
#     axes[1].imshow(perturbed_image_np)
#     axes[1].set_title(f"Adversarial Image {idx+1}, Label: {perturbed_label_idx}")
#     axes[1].axis('off')

#     plt.show()

image count is: 0
image count is: 1
image count is: 2
image count is: 3
image count is: 4
image count is: 5
image count is: 6
image count is: 7
image count is: 8
image count is: 9
image count is: 10
image count is: 11
image count is: 12
image count is: 13
image count is: 14
image count is: 15
image count is: 16
image count is: 17
image count is: 18
image count is: 19
image count is: 20
image count is: 21
image count is: 22
image count is: 23
image count is: 24
image count is: 25
image count is: 26
image count is: 27
image count is: 28
image count is: 29
image count is: 30
image count is: 31
image count is: 32
image count is: 33
image count is: 34
image count is: 35
image count is: 36
image count is: 37
image count is: 38
image count is: 39
image count is: 40
image count is: 41
image count is: 42
image count is: 43
image count is: 44
image count is: 45
image count is: 46
image count is: 47
image count is: 48
image count is: 49
image count is: 50
image count is: 51
image count is: 52
ima

In [3]:
## import os
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt


# Step 1: Load the pre-trained ResNet50 model
model = models.resnet50(pretrained=True)
model.eval()

# Step 2: Load the images
input_directory ="C:/Users/DandaC8719/Documents/capstone_overall_code/output_images_folder(DenseNet)_30"
output_original_directory = "C:/Users/DandaC8719/Documents/capstone_overall_code/original_incorrect_predictions_folder(FGSM)_ResNet_DenseNet"
output_adversarial_directory = "C:/Users/DandaC8719/Documents/capstone_overall_code/adversarial_images_folder(FGSM)_ResNet_DenseNet"

input_images = [os.path.join(input_directory, filename) for filename in os.listdir(input_directory) if filename.endswith(".JPEG")]

# Step 3: Define the FGSM attack function
def fgsm_attack(image, epsilon, data_grad):
    sign_data_grad = data_grad.sign()
    perturbed_image = image + epsilon * sign_data_grad
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    return perturbed_image

# Step 4: Generate adversarial examples
epsilon = 0.01  # Adjust this value as needed
successful_adversarial_examples = []
corres_org_img=[]
count=0
for input_image in input_images[16000:24000]:
    # Load the original image
    original_image = Image.open(input_image).convert("RGB")
    
    # Convert the PIL image to a PyTorch tensor and apply the necessary preprocessing
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ])
    image_tensor = preprocess(original_image).unsqueeze(0)
    image_tensor.requires_grad = True

    # Forward pass to get the predictions
    output = model(image_tensor)
    original_label = torch.argmax(output, 1).item()

    # Calculate the loss
    loss = nn.CrossEntropyLoss()(output, torch.tensor([original_label]))

    # Zero all existing gradients
    model.zero_grad()

    # Backpropagate the loss to compute gradients
    loss.backward()

    # Collect the gradients of the input image
    data_grad = image_tensor.grad.data

    # Generate the perturbed image using the FGSM attack
    perturbed_image = fgsm_attack(image_tensor, epsilon, data_grad)

    # Forward pass with the perturbed image
    perturbed_output = model(perturbed_image)

    # Check if the perturbed image's predicted label differs from the original label
    perturbed_label = torch.argmax(perturbed_output, 1).item()
    if perturbed_label != original_label:
        # Save the perturbed image as a successful adversarial example
        successful_adversarial_examples.append(perturbed_image)
        corres_org_img.append(input_image)
        # Save the original image to output_original_directory
        os.makedirs(output_original_directory, exist_ok=True)
        original_image_name = os.path.basename(input_image)
        output_original_image_path = os.path.join(output_original_directory, original_image_name)
        original_image.save(output_original_image_path)
        
        # Save the perturbed image to output_adversarial_directory
        os.makedirs(output_adversarial_directory, exist_ok=True)
        perturbed_image_name = f"perturbed_{original_image_name}"
        output_perturbed_image_path = os.path.join(output_adversarial_directory, perturbed_image_name)
        perturbed_image_pil = transforms.ToPILImage()(perturbed_image.squeeze(0))
        perturbed_image_pil.save(output_perturbed_image_path)
        print(f"image count is: {count}")
        count +=1

# Print the length of successful adversarial examples
print("Number of successful adversarial examples:", len(successful_adversarial_examples))
print(len(corres_org_img))
print(count)
# # Show the original and adversarial images for all successful examples
# for idx, perturbed_image in enumerate(successful_adversarial_examples):
#     # Load the original image
#     original_image_path = corres_org_img[idx]
#     try:
#         original_image = Image.open(original_image_path).convert("RGB")
#     except Exception as e:
#         print(f"Error loading original image {original_image_path}: {e}")
#         continue

#     # Convert the perturbed image to a numpy array
#     perturbed_image_np = perturbed_image.squeeze().detach().cpu().numpy().transpose(1, 2, 0)

#     # Get the original label
#     with torch.no_grad():
#         original_image_tensor = preprocess(original_image).unsqueeze(0)
#         original_output = model(original_image_tensor)
#         original_label_idx = torch.argmax(original_output, 1).item()

#     # Get the adversarial label
#     with torch.no_grad():
#         perturbed_output = model(perturbed_image)
#         perturbed_label_idx = torch.argmax(perturbed_output, 1).item()

#     fig, axes = plt.subplots(1, 2, figsize=(10, 5))

#     # Plot the original image
#     axes[0].imshow(original_image)
#     axes[0].set_title(f"Original Image {idx+1}, Label: {original_label_idx}")
#     axes[0].axis('off')

#     # Plot the adversarial image
#     axes[1].imshow(perturbed_image_np)
#     axes[1].set_title(f"Adversarial Image {idx+1}, Label: {perturbed_label_idx}")
#     axes[1].axis('off')

#     plt.show()

image count is: 0
image count is: 1
image count is: 2
image count is: 3
image count is: 4
image count is: 5
image count is: 6
image count is: 7
image count is: 8
image count is: 9
image count is: 10
image count is: 11
image count is: 12
image count is: 13
image count is: 14
image count is: 15
image count is: 16
image count is: 17
image count is: 18
image count is: 19
image count is: 20
image count is: 21
image count is: 22
image count is: 23
image count is: 24
image count is: 25
image count is: 26
image count is: 27
image count is: 28
image count is: 29
image count is: 30
image count is: 31
image count is: 32
image count is: 33
image count is: 34
image count is: 35
image count is: 36
image count is: 37
image count is: 38
image count is: 39
image count is: 40
image count is: 41
image count is: 42
image count is: 43
image count is: 44
image count is: 45
image count is: 46
image count is: 47
image count is: 48
image count is: 49
image count is: 50
image count is: 51
image count is: 52
ima

In [4]:
import os
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt


# Step 1: Load the pre-trained ResNet50 model
model = models.resnet50(pretrained=True)
model.eval()

# Step 2: Load the images
input_directory ="C:/Users/DandaC8719/Documents/capstone_overall_code/output_images_folder(DenseNet)_30"
output_original_directory = "C:/Users/DandaC8719/Documents/capstone_overall_code/original_incorrect_predictions_folder(FGSM)_ResNet_DenseNet"
output_adversarial_directory = "C:/Users/DandaC8719/Documents/capstone_overall_code/adversarial_images_folder(FGSM)_ResNet_DenseNet"

input_images = [os.path.join(input_directory, filename) for filename in os.listdir(input_directory) if filename.endswith(".JPEG")]

# Step 3: Define the FGSM attack function
def fgsm_attack(image, epsilon, data_grad):
    sign_data_grad = data_grad.sign()
    perturbed_image = image + epsilon * sign_data_grad
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    return perturbed_image

# Step 4: Generate adversarial examples
epsilon = 0.01  # Adjust this value as needed
successful_adversarial_examples = []
corres_org_img=[]
count=0
for input_image in input_images[24000:]:
    # Load the original image
    original_image = Image.open(input_image).convert("RGB")
    
    # Convert the PIL image to a PyTorch tensor and apply the necessary preprocessing
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ])
    image_tensor = preprocess(original_image).unsqueeze(0)
    image_tensor.requires_grad = True

    # Forward pass to get the predictions
    output = model(image_tensor)
    original_label = torch.argmax(output, 1).item()

    # Calculate the loss
    loss = nn.CrossEntropyLoss()(output, torch.tensor([original_label]))

    # Zero all existing gradients
    model.zero_grad()

    # Backpropagate the loss to compute gradients
    loss.backward()

    # Collect the gradients of the input image
    data_grad = image_tensor.grad.data

    # Generate the perturbed image using the FGSM attack
    perturbed_image = fgsm_attack(image_tensor, epsilon, data_grad)

    # Forward pass with the perturbed image
    perturbed_output = model(perturbed_image)

    # Check if the perturbed image's predicted label differs from the original label
    perturbed_label = torch.argmax(perturbed_output, 1).item()
    if perturbed_label != original_label:
        # Save the perturbed image as a successful adversarial example
        successful_adversarial_examples.append(perturbed_image)
        corres_org_img.append(input_image)
        # Save the original image to output_original_directory
        os.makedirs(output_original_directory, exist_ok=True)
        original_image_name = os.path.basename(input_image)
        output_original_image_path = os.path.join(output_original_directory, original_image_name)
        original_image.save(output_original_image_path)
        
        # Save the perturbed image to output_adversarial_directory
        os.makedirs(output_adversarial_directory, exist_ok=True)
        perturbed_image_name = f"perturbed_{original_image_name}"
        output_perturbed_image_path = os.path.join(output_adversarial_directory, perturbed_image_name)
        perturbed_image_pil = transforms.ToPILImage()(perturbed_image.squeeze(0))
        perturbed_image_pil.save(output_perturbed_image_path)
        print(f"image count is: {count}")
        count +=1

# Print the length of successful adversarial examples
print("Number of successful adversarial examples:", len(successful_adversarial_examples))
print(len(corres_org_img))
print(count)
# # Show the original and adversarial images for all successful examples
# for idx, perturbed_image in enumerate(successful_adversarial_examples):
#     # Load the original image
#     original_image_path = corres_org_img[idx]
#     try:
#         original_image = Image.open(original_image_path).convert("RGB")
#     except Exception as e:
#         print(f"Error loading original image {original_image_path}: {e}")
#         continue

#     # Convert the perturbed image to a numpy array
#     perturbed_image_np = perturbed_image.squeeze().detach().cpu().numpy().transpose(1, 2, 0)

#     # Get the original label
#     with torch.no_grad():
#         original_image_tensor = preprocess(original_image).unsqueeze(0)
#         original_output = model(original_image_tensor)
#         original_label_idx = torch.argmax(original_output, 1).item()

#     # Get the adversarial label
#     with torch.no_grad():
#         perturbed_output = model(perturbed_image)
#         perturbed_label_idx = torch.argmax(perturbed_output, 1).item()

#     fig, axes = plt.subplots(1, 2, figsize=(10, 5))

#     # Plot the original image
#     axes[0].imshow(original_image)
#     axes[0].set_title(f"Original Image {idx+1}, Label: {original_label_idx}")
#     axes[0].axis('off')

#     # Plot the adversarial image
#     axes[1].imshow(perturbed_image_np)
#     axes[1].set_title(f"Adversarial Image {idx+1}, Label: {perturbed_label_idx}")
#     axes[1].axis('off')

#     plt.show()

image count is: 0
image count is: 1
image count is: 2
image count is: 3
image count is: 4
image count is: 5
image count is: 6
image count is: 7
image count is: 8
image count is: 9
image count is: 10
image count is: 11
image count is: 12
image count is: 13
image count is: 14
image count is: 15
image count is: 16
image count is: 17
image count is: 18
image count is: 19
image count is: 20
image count is: 21
image count is: 22
image count is: 23
image count is: 24
image count is: 25
image count is: 26
image count is: 27
image count is: 28
image count is: 29
image count is: 30
image count is: 31
image count is: 32
image count is: 33
image count is: 34
image count is: 35
image count is: 36
image count is: 37
image count is: 38
image count is: 39
image count is: 40
image count is: 41
image count is: 42
image count is: 43
image count is: 44
image count is: 45
image count is: 46
image count is: 47
image count is: 48
image count is: 49
image count is: 50
image count is: 51
image count is: 52
ima